# Summarize by Ward
last updated 3/3/24

<a name="read"></a>
# 1. Read and Prepare Geocoded Fines and 311 Complaints Data

In [1]:
import pandas as pd

In [2]:
df_fines = pd.read_csv("../../data/05-finalized/dockets-summary.csv")
df_fines.head()

,docket,dept,violation_address,lat,long,community,ward_1523,violation_date,n_records
0,19DS68300L,STRTSAN,4710 S WESTERN AVE,41.807859,-87.684797,BRIGHTON PARK,15.0,2019-11-13,2
1,19DS69216L,STRTSAN,1425 W MORSE AVE,42.007451,-87.666828,ROGERS PARK,49.0,2019-11-13,1
2,19DS70010L,STRTSAN,715 E 47TH ST,41.809338,-87.608013,GRAND BOULEVARD,4.0,2019-11-13,1
3,19DS72153L,STRTSAN,300 W WASHINGTON ST,41.881869,-87.740143,WEST GARFIELD PARK,28.0,2019-11-12,5
4,19DS72160L,STRTSAN,6929 N SHERIDAN RD,41.959813,-87.654693,UPTOWN,46.0,2019-11-14,1


In [3]:
df_311 = pd.read_csv("../../data/05-finalized/311-complaints-unshoveled.csv")
df_311.head()

,SR_NUMBER,SR_SHORT_CODE,CREATED_DATE,STREET_ADDRESS,COMMUNITY_AREA,WARD,STATUS,ORIGIN,CLOSED_DATE,LATITUDE,LONGITUDE,SR_TYPE,year,month,date,season,GEOID,COMMUNITY_NAME
0,SR20-05649092,SWSNOREM,2020-12-17 13:24:37,5200 S BLACKSTONE AVE,41,4,Completed,Phone Call,2020-12-18 15:12:51,41.800909,-87.590515,Snow – Uncleared Sidewalk Complaint,2020,12,2020-12-17,2020-2021,41,Hyde Park
1,SR21-00001364,SWSNOREM,2021-01-01 11:40:33,66 E CHESTNUT ST,8,42,Completed,Mobile Device,2021-01-04 14:33:44,41.898399,-87.625723,Snow – Uncleared Sidewalk Complaint,2021,1,2021-01-01,2020-2021,8,Near North Side
2,SR21-00000774,SWSNOREM,2021-01-01 09:11:15,2648 N WHIPPLE ST,22,32,Completed,Internet,2021-01-06 13:37:58,41.930233,-87.703706,Snow – Uncleared Sidewalk Complaint,2021,1,2021-01-01,2020-2021,22,Logan Square
3,SR20-05723378,SWSNOREM,2020-12-31 07:19:37,5015 N SPRINGFIELD AVE,14,39,Completed,Mobile Device,2021-01-08 15:30:57,41.972439,-87.725430,Snow – Uncleared Sidewalk Complaint,2020,12,2020-12-31,2020-2021,14,Albany Park
4,SR21-00014080,SWSNOREM,2021-01-04 14:07:09,3757 N BELL AVE,5,47,Completed,Internet,2021-01-06 11:37:30,41.950316,-87.684472,Snow – Uncleared Sidewalk Complaint,2021,1,2021-01-04,2020-2021,5,North Center


In [4]:
len(df_fines)

1912

In [5]:
len(df_311)

21079

<a name="community"></a>
# Aggregate Dockets by Ward

In [6]:
df_ward_fines = df_fines.pivot_table(index='ward_1523',
                             columns='dept',
                             values='docket',
                             aggfunc=['count'],
                             fill_value=0).reset_index()
# Flatten the MultiIndex in columns
df_ward_fines.columns = ['_'.join(col).strip() for col in df_ward_fines.columns.values]

df_ward_fines = df_ward_fines.rename(columns={'ward_1523_':'ward'})
df_ward_fines.rename(columns=lambda x: x.replace('count_','n_'), inplace=True)

# Reset the index to flatten it
df_ward_fines.reset_index(drop=True, inplace=True)

df_ward_fines['n_dockets'] = df_ward_fines.filter(like='n_').sum(axis=1)
df_ward_fines['ward'] = df_ward_fines['ward'].astype(int)

df_ward_fines.head()

,ward,n_BAFCONP,n_POLICE,n_STRTSAN,n_TRANPORT,n_dockets
0,1,0,0,1,61,62
1,2,0,0,0,64,64
2,3,0,0,0,91,91
3,4,0,0,21,75,96
4,5,0,0,1,7,8


In [7]:
# confirm totals
df_ward_fines['n_dockets'].sum()

1912

In [8]:
len(df_ward_fines)

50

# Aggregate 311 Complaints by Ward

In [9]:
df_ward_311 = df_311.groupby('WARD').size().reset_index(name='n_311_complaints')
df_ward_311.rename(columns={'WARD':'ward'}, inplace = True)
df_ward_311.head()

,ward,n_311_complaints
0,1,1215
1,2,770
2,3,386
3,4,478
4,5,265


In [10]:
df_ward_311['n_311_complaints'].sum()

21079

In [11]:
len(df_ward_311)

50

# Merge Datasets

### Merge Dockets and 311 complaints

In [12]:
df_ward = pd.merge(df_ward_fines,df_ward_311, on = 'ward')
df_ward.head()

,ward,n_BAFCONP,n_POLICE,n_STRTSAN,n_TRANPORT,n_dockets,n_311_complaints
0,1,0,0,1,61,62,1215
1,2,0,0,0,64,64,770
2,3,0,0,0,91,91,386
3,4,0,0,21,75,96,478
4,5,0,0,1,7,8,265


In [13]:
# confirm I have 50 wards
len(df_ward)

50

# Calculations

In [14]:
df_ward['complaint_ratio'] = df_ward['n_TRANPORT']/df_ward['n_311_complaints']

In [15]:
df_ward

,ward,n_BAFCONP,n_POLICE,n_STRTSAN,n_TRANPORT,n_dockets,n_311_complaints,complaint_ratio
0,1,0,0,1,61,62,1215,0.050206
1,2,0,0,0,64,64,770,0.083117
2,3,0,0,0,91,91,386,0.235751
3,4,0,0,21,75,96,478,0.156904
4,5,0,0,1,7,8,265,0.026415
5,6,0,1,53,7,61,180,0.038889
6,7,0,0,6,3,9,107,0.028037
7,8,0,0,0,11,11,228,0.048246
8,9,0,0,1,9,10,90,0.100000
9,10,0,0,0,9,9,126,0.071429


### confirm totals

In [16]:
df_ward.sum()

ward                 1275.000000
n_BAFCONP               2.000000
n_POLICE               25.000000
n_STRTSAN             497.000000
n_TRANPORT           1388.000000
n_dockets            1912.000000
n_311_complaints    21079.000000
complaint_ratio         4.067015
dtype: float64

# Export

In [17]:
df_ward.to_csv("../../results/ssw02-fines/ward-summary.csv", index= False)